In [ ]:
#pip install opensearch-py sentence-transformers tqdm

In [6]:
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------
OPENSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "movies_vector"
VECTOR_FIELD = "plot_vector"
TOP_K = 50

# ----------------------------
# Init OpenSearch client
# ----------------------------
client = OpenSearch(OPENSEARCH_URL)

# ----------------------------
# Load embedding model
# ----------------------------
model = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# Function: vector search
# ----------------------------
def vector_search(query_text, k=TOP_K):
    # 1. Generate query vector
    query_vector = model.encode(query_text).tolist()

    # 2. Build OpenSearch query
    body = {
        "size": k,
        "query": {
            "knn": {
                VECTOR_FIELD: {
                    "vector": query_vector,
                    "k": k
                }
            }
        }
    }

    # 3. Execute search
    response = client.search(
        index=INDEX_NAME,
        body=body
    )
    client

    return response["hits"]["hits"]

# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    query = input("Enter search query: ")

    results = vector_search(query)

    print("\nTop results:\n")
    for hit in results:
        source = hit["_source"]
        print(f"Score: {hit['_score']:.4f}")
        print(f"Title: {source.get('title')}")
        print(f"Plot: {source.get('plot')[:120]}...")

        print("-" * 50)


Enter search query:  romance



Top results:

Score: 0.4816
Title: One Night
Plot:  One Night is a romantic drama about two strangers who meet in a bar and spend the night together. They discover they ha...
--------------------------------------------------
Score: 0.4639
Title: Days of Love
Plot:  The film follows the lives of two young lovers, who are forced to elope because their families disapprove of their rela...
--------------------------------------------------
Score: 0.4584
Title: Eavesdrop
Plot: 1. A couple on a blind date.             2. A chess player and his wife.             3. A man and woman who are having a...
--------------------------------------------------
Score: 0.4561
Title: Las ovejas no pierden el tren
Plot:  The film follows the lives of three couples in their forties who are trying to navigate through the complexities of int...
--------------------------------------------------
Score: 0.4546
Title: I Love, You Love
Plot:  I Love, You Love Pista, a lonely bachelor, working in a Post office, 

<bound method OpenSearch.close of <OpenSearch([{'host': 'localhost', 'port': 9200}])>>

In [2]:
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------
OPENSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "movies_vector"

TITLE_VECTOR_FIELD = "title_vector"
PLOT_VECTOR_FIELD = "plot_vector"

TOP_K = 10

# ----------------------------
# Init clients
# ----------------------------
client = OpenSearch(OPENSEARCH_URL)
model = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# Hybrid search function
# ----------------------------
def hybrid_search(
    query_text,
    genre=None,
    year_range=None,
    rating_range=None,
    duration_range=None,
    likes_range=None,
    size=TOP_K
):
    # ---- Generate query vector ----
    query_vector = model.encode(query_text).tolist()

    # ---- Filters ----
    filters = []

    if genre:
        filters.append({"term": {"genres.keyword": genre}})

    if year_range:
        filters.append({"range": {"year": year_range}})

    if rating_range:
        filters.append({"range": {"rating": rating_range}})

    if duration_range:
        filters.append({"range": {"duration": duration_range}})

    if likes_range:
        filters.append({"range": {"like": likes_range}})

    # ---- Hybrid query ----
    body = {
        "size": size,
        "query": {
            "bool": {
                "filter": filters,
                "should": [
                    # -------- BM25 ----------
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": [
                                "title^3",
                                "plot^1.5",
                                "actors",
                                "directors"
                            ],
                            "type": "best_fields"
                        }
                    },

                    # -------- Title vector ----------
                    {
                        "knn": {
                            TITLE_VECTOR_FIELD: {
                                "vector": query_vector,
                                "k": size,
                                "boost": 3.0
                            }
                        }
                    },

                    # -------- Plot vector ----------
                    {
                        "knn": {
                            PLOT_VECTOR_FIELD: {
                                "vector": query_vector,
                                "k": size,
                                "boost": 1.5
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        }
    }

    response = client.search(
        index=INDEX_NAME,
        body=body
    )

    return response["hits"]["hits"]

# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    results = hybrid_search(
        query_text=" tell me a star wars movie rated 7 plus",
#        genre="Sci-Fi",
#        year_range={"gte": 2000},
#        rating_range={"gte": 7.0},
#        duration_range={"lte": 180},
#        likes_range={"gte": 1000}
    )

    for hit in results:
        src = hit["_source"]
        print("=" * 60)
        print(f"Score   : {hit['_score']:.3f}")
        print(f"Title   : {src.get('title')}")
        print(f"Year    : {src.get('year')}")
        print(f"Rating  : {src.get('rating')}")
        print(f"Genres  : {src.get('genres')}")
        print(f"Plot    : {src.get('plot')[:150]}...")


Score   : 13.622
Title   : Mother Didn't Tell Me
Year    : 1950
Rating  : 4.2
Genres  : Slapstick
Plot    :  A woman learns how to deal with the difficulties of being a doctor's wife, as well as an interfering mother-in-law....
Score   : 11.159
Title   : I Have a Bad Feeling About This
Year    : 2018
Rating  : 4.9
Genres  : Adventure, Parody, Sci-Fi
Plot    :  The plot is based on the Star Wars franchise....
Score   : 9.683
Title   : Kiss Me a Killer
Year    : 1991
Rating  : 4.5
Genres  : Psychological Thriller
Plot    :  A dying night club run by a drunk and his ex-stripper wife comes to life again when a drifter arrives and becomes a singer in the club. An affair beg...
Score   : 8.838
Title   : The Lace Wars
Year    : 1965
Rating  : 2.0
Genres  : Dark Comedy, History, War
Plot    :  The soldier, a young man named Jean, is captured by the enemy and taken to the Prince's castle. There, he is forced to fight in a series of duels aga...
Score   : 8.641
Title   : A Memória que me Contam


In [1]:
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------
OPENSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "movies_vector"

TITLE_VECTOR_FIELD = "title_vector"
PLOT_VECTOR_FIELD = "plot_vector"

TOP_K = 10

# ----------------------------
# Init clients
# ----------------------------
client = OpenSearch(OPENSEARCH_URL)
model = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# Hybrid search function
# ----------------------------
def hybrid_search(
    query_text,
    genre=None,
    year_range=None,
    rating_range=None,
    duration_range=None,
    likes_range=None,
    size=TOP_K
):
    # ---- Generate query vector ----
    query_vector = model.encode(query_text).tolist()

    # ---- Filters ----
    filters = []

    if genre:
        filters.append({"term": {"genres.keyword": genre}})

    if year_range:
        filters.append({"range": {"year": year_range}})

    if rating_range:
        filters.append({"range": {"rating": rating_range}})

    if duration_range:
        filters.append({"range": {"duration": duration_range}})

    if likes_range:
        filters.append({"range": {"like": likes_range}})

    # ---- Hybrid query ----
    body = {
        "size": size,
        "query": {
            "bool": {
                "filter": filters,
                "should": [
                    # -------- BM25 ----------
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": [
                                "title^3",
                                "plot^1.5",
                                "actors",
                                "directors"
                            ],
                            "type": "best_fields"
                        }
                    },

                    # -------- Title vector ----------
                    {
                        "knn": {
                            TITLE_VECTOR_FIELD: {
                                "vector": title_vector,
                                "k": size,
                                "boost": 3.0
                            }
                        }
                    },

                    # -------- Plot vector ----------
                    {
                        "knn": {
                            PLOT_VECTOR_FIELD: {
                                "vector": plot_vector,
                                "k": size,
                                "boost": 1.5
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        }
    }

    response = client.search(
        index=INDEX_NAME,
        body=body
    )

    return response["hits"]["hits"]

# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    results = hybrid_search(
        query_text="suspense",
#        genre="Sci-Fi",
#        year_range={"gte": 2000},
#        rating_range={"gte": 7.0},
#        duration_range={"lte": 180},
#        likes_range={"gte": 1000}
    )

    for hit in results:
        src = hit["_source"]
        print("=" * 60)
        print(f"Score   : {hit['_score']:.3f}")
        print(f"Title   : {src.get('title')}")
        print(f"Year    : {src.get('year')}")
        print(f"Rating  : {src.get('rating')}")
        print(f"Genres  : {src.get('genres')}")
        print(f"Plot    : {src.get('plot')}")


NameError: name 'title_vector' is not defined